In [1]:
import os
import os.path as path
import gc
import re
import math
import json

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_row', 50)
pd.set_option('display.max_columns', 100)

In [3]:
# 디렉토리 기본 경로 설정
DIR_PATH = path.join('..')
DIR_SAVE_PATH = path.join('.')

print(DIR_PATH)
print(DIR_SAVE_PATH)

..
.


In [4]:
# bean_raw_data.csv
bean_read = pd.read_csv(path.join(DIR_PATH, 'bean_raw_data.csv'), low_memory=False)

print(bean_read.shape)
bean_read.head()

(52, 18)


,id,title,origin,region,rank,decaffeination,processing,coffeeing_note,roasting_point,other_note,store,aroma,flavor,acidity,sweetness,bitterness,body,balance
0,1,에티오피아 예가체프 G2,에티오피아,예가체프,G2,-,워시드,"꽃, 과일, 부드러운, 플로럴, 레몬, 허니",중배전,"핸드드립, 모카포트",커피창고,7,7,7,6,4,4,-
1,2,과테말라 안티구아,과테말라,안티구아,SHB,-,워시드,"스모크, 우아한, 중후한, 블랙커런트, 갈색설탕, 다크초코",강배전,"핸드드립, 모카포트",커피창고,6,6,3,7,7,7,-
2,3,에티오피아 코케허니 G1,에티오피아,코케,G1,-,펄프드내추럴,"베리, 체리, 허니",중배전,"핸드드립, 모카포트",커피창고,8,8,8,7,4,5,-
3,4,케냐 AA,케냐,키암부,AA,-,워시드,"자몽, 당밀, 카라멜",중배전,"핸드드립, 모카포트",커피창고,6,6,6,7,6,7,-
4,5,콜롬비아 수프리모,콜롬비아,콜롬비아,SUPREMO,-,워시드,"마일드, 적포도, 메이플시럽, 다크초코",중배전,"핸드드립, 모카포트",커피창고,7,7,5,6,6,6,-


In [5]:
bean_data = bean_read.copy()
bean_data.drop(labels=['id', 'title'], axis=1, inplace=True)
bean_data.head()

,origin,region,rank,decaffeination,processing,coffeeing_note,roasting_point,other_note,store,aroma,flavor,acidity,sweetness,bitterness,body,balance
0,에티오피아,예가체프,G2,-,워시드,"꽃, 과일, 부드러운, 플로럴, 레몬, 허니",중배전,"핸드드립, 모카포트",커피창고,7,7,7,6,4,4,-
1,과테말라,안티구아,SHB,-,워시드,"스모크, 우아한, 중후한, 블랙커런트, 갈색설탕, 다크초코",강배전,"핸드드립, 모카포트",커피창고,6,6,3,7,7,7,-
2,에티오피아,코케,G1,-,펄프드내추럴,"베리, 체리, 허니",중배전,"핸드드립, 모카포트",커피창고,8,8,8,7,4,5,-
3,케냐,키암부,AA,-,워시드,"자몽, 당밀, 카라멜",중배전,"핸드드립, 모카포트",커피창고,6,6,6,7,6,7,-
4,콜롬비아,콜롬비아,SUPREMO,-,워시드,"마일드, 적포도, 메이플시럽, 다크초코",중배전,"핸드드립, 모카포트",커피창고,7,7,5,6,6,6,-


In [6]:
# 카페인 결측치 채우기
bean_data.replace({'decaffeination':'-'}, 'F', inplace=True)
bean_data.head(1)

,origin,region,rank,decaffeination,processing,coffeeing_note,roasting_point,other_note,store,aroma,flavor,acidity,sweetness,bitterness,body,balance
0,에티오피아,예가체프,G2,F,워시드,"꽃, 과일, 부드러운, 플로럴, 레몬, 허니",중배전,"핸드드립, 모카포트",커피창고,7,7,7,6,4,4,-


In [7]:
# 숫자 결측치 채우기
# 단순하게 측정치의 중앙인 5로 결측치 보간
bean_data.replace({'aroma':'-', 'flavor':'-', 'acidity':'-', 'sweetness':'-', 'bitterness':'-', 'body':'-', 'balance':'-'}, '5', inplace=True)
bean_data = bean_data.astype({'aroma':'int', 'flavor':'int', 'acidity':'int', 'sweetness':'int', 'bitterness':'int', 'body':'int', 'balance':'int'})
bean_data.head(2)

,origin,region,rank,decaffeination,processing,coffeeing_note,roasting_point,other_note,store,aroma,flavor,acidity,sweetness,bitterness,body,balance
0,에티오피아,예가체프,G2,F,워시드,"꽃, 과일, 부드러운, 플로럴, 레몬, 허니",중배전,"핸드드립, 모카포트",커피창고,7,7,7,6,4,4,5
1,과테말라,안티구아,SHB,F,워시드,"스모크, 우아한, 중후한, 블랙커런트, 갈색설탕, 다크초코",강배전,"핸드드립, 모카포트",커피창고,6,6,3,7,7,7,5


In [8]:
# aroma와 flavor합치기 => flavor만 남기기
# aroma + flavor / 2
bean_data['flavor'] = round((bean_data['flavor'] + bean_data['aroma']) / 2)
bean_data.drop(labels=['aroma'], axis=1, inplace=True)
bean_data = bean_data.astype({'flavor':'int'})
bean_data.head(2)

,origin,region,rank,decaffeination,processing,coffeeing_note,roasting_point,other_note,store,flavor,acidity,sweetness,bitterness,body,balance
0,에티오피아,예가체프,G2,F,워시드,"꽃, 과일, 부드러운, 플로럴, 레몬, 허니",중배전,"핸드드립, 모카포트",커피창고,7,7,6,4,4,5
1,과테말라,안티구아,SHB,F,워시드,"스모크, 우아한, 중후한, 블랙커런트, 갈색설탕, 다크초코",강배전,"핸드드립, 모카포트",커피창고,6,3,7,7,7,5


In [9]:
bean_data['flavor'].value_counts(dropna=False)

8    19
6    18
7     8
4     5
5     1
9     1
Name: flavor, dtype: int64

In [10]:
bean_data.columns

Index(['origin', 'region', 'rank', 'decaffeination', 'processing',
       'coffeeing_note', 'roasting_point', 'other_note', 'store', 'flavor',
       'acidity', 'sweetness', 'bitterness', 'body', 'balance'],
      dtype='object')

In [11]:
bean_data['title'] = bean_data['origin'] + ' ' \
    + bean_data['region'] + ' ' + bean_data['rank'] + ' ' \
    + bean_data['processing'] \
    + np.where(bean_data['decaffeination'] == 'T', ' 디카페인' , '') \
    + ' (' + bean_data['roasting_point'] + ')'
bean_data = bean_data[['title', 'origin', 'region', 'rank', 'decaffeination', 'processing',
       'coffeeing_note', 'roasting_point', 'other_note', 'store', 'flavor',
       'acidity', 'sweetness', 'bitterness', 'body', 'balance']]
bean_data.head(2)

,title,origin,region,rank,decaffeination,processing,coffeeing_note,roasting_point,other_note,store,flavor,acidity,sweetness,bitterness,body,balance
0,에티오피아 예가체프 G2 워시드 (중배전),에티오피아,예가체프,G2,F,워시드,"꽃, 과일, 부드러운, 플로럴, 레몬, 허니",중배전,"핸드드립, 모카포트",커피창고,7,7,6,4,4,5
1,과테말라 안티구아 SHB 워시드 (강배전),과테말라,안티구아,SHB,F,워시드,"스모크, 우아한, 중후한, 블랙커런트, 갈색설탕, 다크초코",강배전,"핸드드립, 모카포트",커피창고,6,3,7,7,7,5


In [12]:
bean_data.loc[bean_data['title'] == '에티오피아 시다모 G2 워시드 (중배전)']

,title,origin,region,rank,decaffeination,processing,coffeeing_note,roasting_point,other_note,store,flavor,acidity,sweetness,bitterness,body,balance
23,에티오피아 시다모 G2 워시드 (중배전),에티오피아,시다모,G2,F,워시드,"레드와인, 레몬, 코코아",중배전,"핸드드립, 모카포트",커피창고,7,6,6,5,6,5
35,에티오피아 시다모 G2 워시드 (중배전),에티오피아,시다모,G2,F,워시드,"과일, 플로럴, 산뜻한, 감칠맛,",중배전,-,다크브릭스,8,6,6,5,4,6


In [13]:
## 빠른 데이터 추가를 위해서 중복인 경우 중복 행을 Drop
## 나중에는 중복 데이터의 경우 note는 set으로 유니크한 속성만 남기고, status는 평균 값을 사용
bean_data.drop_duplicates(subset=['title'], keep='first', inplace=True)
bean_data['title'].value_counts(dropna=False)

에티오피아 예가체프 G2 워시드 (중배전)             1
탄자니아 음빙가 AA 워시드 (강배전)               1
콜롬비아 콜롬비아 SUPREMO 워시드 디카페인 (강배전)    1
에티오피아 예가체프 G2 워시드 (약배전)             1
브라질 세하도 NY2 FC 네추럴 (중배전)            1
과테말라 안티구아 SHB 워시드 (중배전)             1
콜롬비아 후일라 SUPREMO 워시드 (중배전)          1
베트남 다낭 G1 폴리싱 (강배전)                 1
브라질 산토스 NY2 FC 내추럴 (중배전)            1
에티오피아 시다모 G4 내추럴 (중배전)              1
에티오피아 예가체프 G4 내추럴 (중배전)             1
콜롬비아 메데인 SUPREMO 워시드 (강배전)          1
인도 치크마갈루르 AA 워시드 (중배전)              1
코스타리카 따라주 SHB 워시드 (강배전)             1
파푸아뉴기니 와기벨리 AA 워시드 (강배전)            1
인도네시아 만델링 G1 워시드 (강배전)              1
온두라스 산티아고 푸링글라 SHB 워시드 (강배전)        1
엘 살바도르 엘 살바도르 SHG 워시드 (강배전)         1
케냐 키암부 AA 워시드 (강배전)                 1
콜롬비아 메데인 SUPREMO 워시드 디카페인 (강배전)     1
과테말라 안티구아 SHB 워시드 디카페인 (강배전)        1
에티오피아 예가체프 G2 워시드 디카페인 (강배전)        1
에티오피아 시다모 G2 워시드 디카페인 (중배전)         1
에티오피아 시다모 G2 워시드 (중배전)              1
과테말라 안티구아 SHB 워시드 (강배전)             1
코스타리카 나란조 MICRO LOT 워시드 (중배전)       1
에티오피아 코케 G1 

In [14]:
# 인덱스 재지정
bean_data.reset_index(drop=True, inplace=True)
bean_data

,title,origin,region,rank,decaffeination,processing,coffeeing_note,roasting_point,other_note,store,flavor,acidity,sweetness,bitterness,body,balance
0,에티오피아 예가체프 G2 워시드 (중배전),에티오피아,예가체프,G2,F,워시드,"꽃, 과일, 부드러운, 플로럴, 레몬, 허니",중배전,"핸드드립, 모카포트",커피창고,7,7,6,4,4,5
1,과테말라 안티구아 SHB 워시드 (강배전),과테말라,안티구아,SHB,F,워시드,"스모크, 우아한, 중후한, 블랙커런트, 갈색설탕, 다크초코",강배전,"핸드드립, 모카포트",커피창고,6,3,7,7,7,5
2,에티오피아 코케 G1 펄프드내추럴 (중배전),에티오피아,코케,G1,F,펄프드내추럴,"베리, 체리, 허니",중배전,"핸드드립, 모카포트",커피창고,8,8,7,4,5,5
3,케냐 키암부 AA 워시드 (중배전),케냐,키암부,AA,F,워시드,"자몽, 당밀, 카라멜",중배전,"핸드드립, 모카포트",커피창고,6,6,7,6,7,5
4,콜롬비아 콜롬비아 SUPREMO 워시드 (중배전),콜롬비아,콜롬비아,SUPREMO,F,워시드,"마일드, 적포도, 메이플시럽, 다크초코",중배전,"핸드드립, 모카포트",커피창고,7,5,6,6,6,5
5,페루 피우라 MICRO LOT 워시드 (중배전),페루,피우라,MICRO LOT,F,워시드,"사과, 자두, 살구, 스카치 캔디, 조청, 밀크초코",중배전,"핸드드립, 모카포트",커피창고,8,6,8,4,8,5
6,브라질 세하도 NY2 FC 내추럴 (중배전),브라질,세하도,NY2 FC,F,내추럴,"카라멜, 산뜻한, 자연스러운, 가벼운, 건포도, 땅콩",중배전,"핸드드립, 모카포트",커피창고,5,5,6,5,6,5
7,케냐 키암부 FAQ 워시드 (중배전),케냐,키암부,FAQ,F,워시드,"오렌지, 시트러스, 아몬드, 땅콩, 조청, 갈색설탕",중배전,"핸드드립, 모카포트",커피창고,6,6,6,6,6,5
8,파나마 보케테 SPECIALTY 워시드 (중배전),파나마,보케테,SPECIALTY,F,워시드,"가벼운, 균형잡힌, 풍부한, 헤이즐넛, 바닐라, 카카오",중배전,"핸드드립, 모카포트",커피창고,7,5,7,7,7,5
9,브라질 세하도 NY2 FC 펄프드내추럴 (중배전),브라질,세하도,NY2 FC,F,펄프드내추럴,"균형잡힌, 고소한, 피넛, 시트러스, 쿠키",중배전,"핸드드립, 모카포트",커피창고,7,5,7,6,6,5


In [15]:
bean_data.columns

Index(['title', 'origin', 'region', 'rank', 'decaffeination', 'processing',
       'coffeeing_note', 'roasting_point', 'other_note', 'store', 'flavor',
       'acidity', 'sweetness', 'bitterness', 'body', 'balance'],
      dtype='object')

In [16]:
sql_bean = pd.DataFrame()
sql_bean['idx'] = bean_data.index + 1
sql_bean['created_date'] = ""
sql_bean['updated_date'] = ""
sql_bean['name_ko'] = bean_data['title']
sql_bean['name_en'] = bean_data['title']
sql_bean['summary'] = "아이템 요약 영역 입니다!!!"
sql_bean['thumbnail'] = "default_bean.png"
sql_bean['user_grade'] = 0
sql_bean.head()

,idx,created_date,updated_date,name_ko,name_en,summary,thumbnail,user_grade
0,1,,,에티오피아 예가체프 G2 워시드 (중배전),에티오피아 예가체프 G2 워시드 (중배전),아이템 요약 영역 입니다!!!,default_bean.png,0
1,2,,,과테말라 안티구아 SHB 워시드 (강배전),과테말라 안티구아 SHB 워시드 (강배전),아이템 요약 영역 입니다!!!,default_bean.png,0
2,3,,,에티오피아 코케 G1 펄프드내추럴 (중배전),에티오피아 코케 G1 펄프드내추럴 (중배전),아이템 요약 영역 입니다!!!,default_bean.png,0
3,4,,,케냐 키암부 AA 워시드 (중배전),케냐 키암부 AA 워시드 (중배전),아이템 요약 영역 입니다!!!,default_bean.png,0
4,5,,,콜롬비아 콜롬비아 SUPREMO 워시드 (중배전),콜롬비아 콜롬비아 SUPREMO 워시드 (중배전),아이템 요약 영역 입니다!!!,default_bean.png,0


In [17]:
# 파일 저장
os.makedirs(DIR_SAVE_PATH, exist_ok=True)
sql_bean.to_csv(path.join(DIR_SAVE_PATH, 'sql_bean.csv'), sep=',', index=False, encoding='utf-8')

In [18]:
sql_bean_detail = pd.DataFrame()
sql_bean_detail['idx'] = bean_data.index + 1
sql_bean_detail['created_date'] = ""
sql_bean_detail['updated_date'] = ""
sql_bean_detail['decaffeination'] = bean_data['decaffeination']
sql_bean_detail['description'] = "상세 내용 입니다!!!"
sql_bean_detail['origin'] = bean_data['origin']
sql_bean_detail['region'] = bean_data['region']
sql_bean_detail['rank'] = bean_data['rank']
sql_bean_detail['processing'] = bean_data['processing']
sql_bean_detail['bean_idx'] = bean_data.index + 1
sql_bean_detail.head()

,idx,created_date,updated_date,decaffeination,description,origin,region,rank,processing,bean_idx
0,1,,,F,상세 내용 입니다!!!,에티오피아,예가체프,G2,워시드,1
1,2,,,F,상세 내용 입니다!!!,과테말라,안티구아,SHB,워시드,2
2,3,,,F,상세 내용 입니다!!!,에티오피아,코케,G1,펄프드내추럴,3
3,4,,,F,상세 내용 입니다!!!,케냐,키암부,AA,워시드,4
4,5,,,F,상세 내용 입니다!!!,콜롬비아,콜롬비아,SUPREMO,워시드,5


In [19]:
# 파일 저장
os.makedirs(DIR_SAVE_PATH, exist_ok=True)
sql_bean_detail.to_csv(path.join(DIR_SAVE_PATH, 'sql_bean_detail.csv'), sep=',', index=False, encoding='utf-8')

In [20]:
sql_bean_score = pd.DataFrame()
sql_bean_score['idx'] = bean_data.index + 1
sql_bean_score['created_date'] = ""
sql_bean_score['updated_date'] = ""
sql_bean_score['flavor'] = bean_data['flavor']
sql_bean_score['acidity'] = bean_data['acidity']
sql_bean_score['sweetness'] = bean_data['sweetness']
sql_bean_score['bitterness'] = bean_data['bitterness']
sql_bean_score['body'] = bean_data['body']
sql_bean_score['balance'] = bean_data['balance']
sql_bean_score['roasting_point'] = bean_data['roasting_point']
sql_bean_score['coffeeing_note'] = bean_data['coffeeing_note']
sql_bean_score['bean_idx'] = bean_data.index + 1
sql_bean_score.head()

,idx,created_date,updated_date,flavor,acidity,sweetness,bitterness,body,balance,roasting_point,coffeeing_note,bean_idx
0,1,,,7,7,6,4,4,5,중배전,"꽃, 과일, 부드러운, 플로럴, 레몬, 허니",1
1,2,,,6,3,7,7,7,5,강배전,"스모크, 우아한, 중후한, 블랙커런트, 갈색설탕, 다크초코",2
2,3,,,8,8,7,4,5,5,중배전,"베리, 체리, 허니",3
3,4,,,6,6,7,6,7,5,중배전,"자몽, 당밀, 카라멜",4
4,5,,,7,5,6,6,6,5,중배전,"마일드, 적포도, 메이플시럽, 다크초코",5


In [21]:
# 파일 저장
os.makedirs(DIR_SAVE_PATH, exist_ok=True)
sql_bean_score.to_csv(path.join(DIR_SAVE_PATH, 'sql_bean_score.csv'), sep=',', index=False, encoding='utf-8')